# CS460 Algorithms and Their Analysis
## Programming Assignment 7: Advanced graph algorithms -- Part 1, Minimum spanning tree

**Author:** Yang Xu, Assistant Professor of Computer Science, San Diego State University

**Total points: 10**

In [107]:
from sys import maxsize 

## Task 1. Import pre-defined classes

**Points:1**

Import `Edge` class, and run the following to create a list of edges.

In [108]:
from mst_util import Edge

def create_examples():
    """
    Create a list of edges that will be used in the assignment tasks
    :return:
    """
    vertices_string = '0 1 4\n' \
                      '1 2 8\n' \
                      '2 3 7\n' \
                      '3 4 9\n' \
                      '4 5 10\n' \
                      '5 6 2\n' \
                      '6 7 1\n' \
                      '7 8 7\n' \
                      '0 7 8\n' \
                      '1 7 11\n' \
                      '2 8 2\n' \
                      '6 8 6\n' \
                      '2 5 4\n' \
                      '3 5 14'

    edges = []
    vertices = []
    for item in vertices_string.split('\n'):
        s, t, w = map(int, item.split())
        edge = Edge(s, t, w)
        edges.append(edge)
        if s not in vertices:
            vertices.append(s)
        if t not in vertices:
            vertices.append(t)

    return edges, vertices

In [109]:
edge1 = Edge('u', 'v', 2)
edge2 = Edge('s', 't', 3)

print(edge1)
print(edge2)

edges, vertices = create_examples()
print('edges:', edges)
print('vertices:', vertices)

e = edges[0]
print(e.source)
print(e.target)
print(e.weight)

(u--2--v)
(s--3--t)
edges: [(0--4--1), (1--8--2), (2--7--3), (3--9--4), (4--10--5), (5--2--6), (6--1--7), (7--7--8), (0--8--7), (1--11--7), (2--2--8), (6--6--8), (2--4--5), (3--14--5)]
vertices: [0, 1, 2, 3, 4, 5, 6, 7, 8]
0
1
4


As you can see above, you can access the `.source`, `.target`, `.weight` attributes of an edge.

Import `DisjointSet` and play with the following code.

The disjoint set is implemented as a list `n` of integers, in which `n[i]` is the parent of the node at position `i`. If `n[i] == i`, it means node `i` is the root (or representative) of a set.

In the following examples, `merge_set()` has the same function as the UNION() in the pseudo-code of textbook.

In [110]:
# Do not change the test code here
from mst_util import DisjointSet

test_set = DisjointSet(5)
print('In initial set')
print(f'parent of 1 is {test_set.find_set(1)}')
print(f'parent of 4 is {test_set.find_set(4)}')

test_set.merge_set(0,1)
test_set.merge_set(3,4)
print()
print('After merging (0,1) and (3,4)')
print(f'parent of 1 is {test_set.find_set(1)}')
print(f'parent of 4 is {test_set.find_set(4)}')

print('\nCheck if 0 and 3 are in the same set:')
print(test_set.find_set(0) == test_set.find_set(3))

In initial set
parent of 1 is 1
parent of 4 is 4

After merging (0,1) and (3,4)
parent of 1 is 0
parent of 4 is 3

Check if 0 and 3 are in the same set:
False


---

## Task 2. Implement Kruskal algorithm

**Points: 4**

Use the above imported data structure to implement Kruskal algorithm.

Note that you can use the built-in `sorted` function (or the `.sort()` of list object to carry out an in-place sort) to sort the list of edges, based on their weights. *Hint*: can speficy the `key` parameter with a `lambda` statement.

In [111]:
def kruskal_mst(edges, vertex_count):
    """
    :param edges: edges (list of Edge): Edges of the graph
    :param vertex_count: vertex_count (int): Number of vertices in the graph
    :return: the MST formed by a list of edges
    """
    # Initialize the disjoint set
    ### START YOUR CODE ###
    forest = DisjointSet(vertex_count) # number of vertices in the graph
    ### END YOUR CODE ###

    # Sort edges based on weight
    ### START YOUR CODE ###
    edges = sorted(edges, key=lambda x: x.weight)
    ### END YOUR CODE ###

    mst = [] # List of edges taken, i.e., the minimum spanning tree

    for edge in edges:
        ### START YOUR CODE ###
        set_u = forest.find_set(edge.source) # Find the set of one vertex
        set_v = forest.find_set(edge.target) # Find the set of the other vertex
        if set_u != set_v:
            forest.merge_set(set_u, set_v) # Merge (union) the two sets

            mst.append(Edge(edge.source, edge.target, edge.weight)) # Add the edge to mst

            # Add a condition for early termination
            if len(mst) == vertex_count-1:
                break
        ### END YOUR CODE ###

    return mst

In [112]:
# Do not change the test code here
edges, vertices = create_examples()
mst = kruskal_mst(edges, len(vertices))

print(mst)

[(6--1--7), (5--2--6), (2--2--8), (0--4--1), (2--4--5), (2--7--3), (1--8--2), (3--9--4)]


**Expected output**:

[(6--1--7), (5--2--6), (2--2--8), (0--4--1), (2--4--5), (2--7--3), (1--8--2), (3--9--4)]

---

## Task 3. Implement Prim's algorithm
**Points: 5**

Implement Prim's algorithm by using the minimum priority queue and undirected weighted graph data structure defined in `prims_utils.py`.
Make sure `prims_utils.py` is within the same directory of this notebook.

First, use the following examples to get familar with how to use the two classes.

Check out the following usage of `push()` function, which allows you to enqueue elements with specified weights; `has()` to examine if element exists in the queue; `extract_min()` to extract the element with minimum key; and `update_key()` to manaully update the key of an element.

In [113]:
# Do not change the test code below
from mst_util import MinPriorityQueue, GraphUndirectedWeighted

# Uses of MinPriorityQueue
# Add elements
test_pq = MinPriorityQueue()
test_pq.push(1, 1000)
test_pq.push(2, 100)
test_pq.push(3, 4000)
test_pq.push(4, 3000)
print(test_pq)

# Examine if element exists
print(test_pq.has(1))

# Extract minimum
print(test_pq.extract_min())

# Update key
test_pq.update_key(4, 50) # 3000 => 50
print(test_pq)

[(2, 100), (1, 1000), (3, 4000), (4, 3000)]
True
2
[(4, 50), (1, 1000), (3, 4000)]


Next, play with the `GraphUndirectedWeighted` class.

In [114]:
# Do not change the test code below
test_graph = GraphUndirectedWeighted()

# Examples for adding weighted edges
test_graph.add_edge('a', 'b', 4)
test_graph.add_edge('b', 'c', 8)

# Access the adjacency lists
print(type(test_graph.connections))
print(test_graph.connections)

<class 'dict'>
{'a': {'b': 4}, 'b': {'a': 4, 'c': 8}, 'c': {'b': 8}}


As you can see, the adjacency lists of the graph is stored in the `dict` object `connections`. The keys are vertices, and each value is a sub `dict` containing the adjacent vertices. The values of this each sub `dict` are the edge weights.

That means you can access the weight between vertice `u` and `v` by `connections[u][v]`

Now, you can implement the Prim's algorithm.

Note that when you intialize the `keys` object, use `maxsize` as the infinity, which is imported in the first cell of notebook.

`keys` and `parents` are two `dict` objects, whose keys are the vertices of the graph.

In [143]:
def prims_mst(graph: GraphUndirectedWeighted, root):
    # Initialize the priority queue
    pq = MinPriorityQueue()

    # Initialize the `keys` and `parents` dict objects
    ### START YOUR CODE ###
    keys = dict.fromkeys(graph.connections.keys(), maxsize)
    parents = dict.fromkeys(graph.connections.keys()) # values will be null
    ### END YOUR CODE ###
    total_weight = 0

    # Initialize the root vertex's key
    ### START YOUR CODE ###
    keys[root] = 0
    ### END YOUR CODE ###

    # Push all (vertice, infinity) pairs to the queue
    ### START YOUR CODE ###
    for key in keys: # Hint: User a for loop over `keys` object
        pq.push(key, maxsize)
    ### END YOUR CODE ###

    # Loop
    while not pq.is_empty():
        ### START YOUR CODE ###
        u = pq.extract_min() # Extract minimum from the pq
        total_weight += keys[u] # Add its key to total_weight

        for v in graph.connections[u]: # Iterate over adjacent vertices
            if pq.has(v) and graph.connections[u][v] < keys[v]: # Specify the condition
                parents[v] = u # Update parent
                keys[v] = graph.connections[u][v] # Update the key of v in `keys`
                pq.update_key(v, keys[v]) # Manually update the position of v in pq
        ### END YOUR CODE ###

    return parents, total_weight

In [144]:
# Do not change the test code here
graph = GraphUndirectedWeighted()
graph.add_edge('a', 'b', 4)
graph.add_edge('b', 'c', 8)
graph.add_edge('c', 'd', 7)
graph.add_edge('d', 'e', 9)
graph.add_edge('e', 'f', 10)
graph.add_edge('d', 'f', 14)
graph.add_edge('c', 'f', 4)
graph.add_edge('f', 'g', 2)
graph.add_edge('c', 'i', 2)
graph.add_edge('g', 'i', 6)
graph.add_edge('g', 'h', 1)
graph.add_edge('h', 'i', 7)
graph.add_edge('h', 'b', 11)
graph.add_edge('h', 'a', 8)

parents, total_weight = prims_mst(graph, 'a')

print(parents)
print(total_weight)

{'a': None, 'b': 'a', 'c': 'f', 'd': 'c', 'e': 'd', 'f': 'g', 'g': 'h', 'i': 'c', 'h': 'a'}
37


**Expected output**

{'a': None, 'b': 'a', 'c': 'f', 'd': 'c', 'e': 'd', 'f': 'g', 'g': 'h', 'i': 'c', 'h': 'a'}
37